In [2]:
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_features.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/submission_format.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/training_set_labels.csv
!wget https://s3.amazonaws.com/drivendata-prod/data/66/public/test_set_features.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


## Importing the needed libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  f1_score
from sklearn.impute import SimpleImputer



%matplotlib inline


## Importing data

In [6]:
path_train_X='./training_set_features.csv'
path_train_Y='./training_set_labels.csv'
path_test_X='./test_set_features.csv'

In [7]:
train_set_X = pd.read_csv(path_train_X)
print(train_set_X.head())

   respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0              0           1.0             0.0                        0.0   
1              1           3.0             2.0                        0.0   
2              2           1.0             1.0                        0.0   
3              3           1.0             1.0                        0.0   
4              4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0               

In [8]:
train_set_Y = pd.read_csv(path_train_Y)
print(train_set_Y.head())
print(train_set_Y.shape)

   respondent_id  h1n1_vaccine  seasonal_vaccine
0              0             0                 0
1              1             0                 1
2              2             0                 0
3              3             0                 1
4              4             0                 0
(26707, 3)


In [9]:
test_set_X = pd.read_csv(path_test_X)
test_set_X.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [10]:
train_set_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
respondent_id                  26707 non-null int64
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective  

In [11]:
test_set_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26708 entries, 0 to 26707
Data columns (total 36 columns):
respondent_id                  26708 non-null int64
h1n1_concern                   26623 non-null float64
h1n1_knowledge                 26586 non-null float64
behavioral_antiviral_meds      26629 non-null float64
behavioral_avoidance           26495 non-null float64
behavioral_face_mask           26689 non-null float64
behavioral_wash_hands          26668 non-null float64
behavioral_large_gatherings    26636 non-null float64
behavioral_outside_home        26626 non-null float64
behavioral_touch_face          26580 non-null float64
doctor_recc_h1n1               24548 non-null float64
doctor_recc_seasonal           24548 non-null float64
chronic_med_condition          25776 non-null float64
child_under_6_months           25895 non-null float64
health_worker                  25919 non-null float64
health_insurance               14480 non-null float64
opinion_h1n1_vacc_effective  

## Merging the two datasets

In [12]:
train_set = pd.DataFrame(pd.concat((train_set_X,train_set_Y[['h1n1_vaccine' , 'seasonal_vaccine']]),axis=1))
train_set['respondent_id'] = train_set['respondent_id'].astype(str)
test_set_X['respondent_id'] = test_set_X['respondent_id'].astype(str)
print(train_set.head())


  respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0             0           1.0             0.0                        0.0   
1             1           3.0             2.0                        0.0   
2             2           1.0             1.0                        0.0   
3             3           1.0             1.0                        0.0   
4             4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0                     

## Imputing number columns

In [ ]:
train_set_number = train_set.select_dtypes(exclude='object')
test_set_number = test_set_X.select_dtypes(exclude='object')

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_set = pd.DataFrame(imp.fit_transform(train_set))

In [ ]:
train_set.info(),test_set_X.info()

## Dealing with non numerical data

#### Droping columns with too many data missing

In [ ]:
sums = train_set.isna().sum()
length = train_set.shape[0]
to_drop = sums[sums > 0.49 * length].keys()
print(to_drop)

In [ ]:
train_set = train_set.drop(columns=list(to_drop))
test_set_X = test_set_X.drop(columns=list(to_drop))

In [ ]:
train_set.shape,test_set_X.shape

In [ ]:
train_set.select_dtypes('object').isna().sum() ,test_set_X.select_dtypes('object').isna().sum() 

#### Keeping the others and imputing them:

In [ ]:
sums = train_set.select_dtypes('object').isna().sum() 
cols = list(sums[sums > 0].keys())
cols

In [ ]:
train_set.dropna().shape

In [ ]:
df = train_set.dropna()

In [ ]:
df['employment_status'].value_counts()

## Splitting into X and Y and train and valid for h1n1

In [ ]:
X = df.drop(columns=['h1n1_vaccine' ,'respondent_id' , 'seasonal_vaccine'])
col_to_keep = [col for col in list(X.columns)]
X = pd.get_dummies(X, columns=col_to_keep)
test_X = pd.get_dummies(test_set_X.drop(columns=['respondent_id']),prefix='C',columns=col_to_keep)
Y_h1n1 = df[['h1n1_vaccine' ]]
Y_seasonal = df[['seasonal_vaccine']]
print(X.head())
print(Y.head())
len(test_X.columns),len(X.columns)

In [ ]:
X.columns

In [ ]:
X_train,X_valid,Y_train_h1n1,Y_valid_h1n1 = train_test_split(X,Y_h1n1,test_size=0.1,random_state=101,stratify=Y_h1n1.values)

In [ ]:
Y_train_h1n1.head()

## Training a logistic regression on this model

In [ ]:
logreg = LogisticRegression(random_state=101,max_iter=400)
logreg.fit(X_train,Y_train_h1n1.to_numpy().ravel())
logreg.score(X_train,Y_train_h1n1)*100,logreg.score(X_valid,Y_valid_h1n1)*100

In [ ]:
test_Y = logreg.predict(test_X)
test_Y[:10]

## Keeping only important features for h1n1

In [ ]:
logreg.coef_

In [ ]:
mask = (abs(logreg.coef_) > 0.05).ravel().reshape(-1,1)
cols_h1n1 = np.array(X_train.columns).reshape(-1,1)[mask]

In [ ]:
X_train_engineered = X_train.loc[:,cols_h1n1]
X_valid_engineered = X_valid.loc[:,cols_h1n1]
X_train_engineered.columns

In [ ]:
logreg2 = LogisticRegression(max_iter=400)
logreg2.fit(X_train_engineered,Y_train_h1n1.to_numpy().ravel())
print(logreg2.score(X_train_engineered,Y_train_h1n1)*100)
print(logreg2.score(X_valid_engineered,Y_valid_h1n1)*100)

In [ ]:
test_X[cols_h1n1]

## Trying a more compilcated model

In [ ]:
linsvc = LinearSVC(random_state=101,max_iter=1500)
linsvc.fit(X_train,Y_train_h1n1.to_numpy().ravel())
linsvc.score(X_train,Y_train_h1n1)*100,linsvc.score(X_valid,Y_valid_h1n1)*100

In [ ]:
svc = SVC(random_state=101,max_iter=1500,kernel='rbf')
svc.fit(X_train,Y_train_h1n1.to_numpy().ravel())
svc.score(X_train,Y_train_h1n1)*100,svc.score(X_valid,Y_valid_h1n1)*100

In [ ]:
rfc = RandomForestClassifier(random_state=101,n_estimators=600)
rfc.fit(X_train,Y_train_h1n1.to_numpy().ravel())
rfc.score(X_train,Y_train_h1n1)*100,rfc.score(X_valid,Y_valid_h1n1)*100

In [ ]:
f1_score(logreg.predict(X_train),Y_train_h1n1)